**Installation de la cli et l'api de dvc**

In [ ]:
!pip install dvc dvc-s3
!curl -sLO https://github.com/iterative/dvc/releases/download/2.35.1/dvc_2.35.1_amd64.deb
!sudo apt install ./dvc_2.35.1_amd64.deb
!rm ./dvc_2.35.1_amd64.deb
!echo "Installed with success!"
!dvc --version

On commence avec un dépôt Git vide dans lequel on initialise dvc avec la commande ```dvc init```. Cela crée un dossier ```.dvc``` équivalent à ```.git```.

In [2]:
!git clone https://github.com/amine-bs/data-version-control.git
%cd data-version-control

Cloning into 'data-version-control'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 653 bytes | 163.00 KiB/s, done.
/home/onyxia/work/data-version-control


In [3]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
!git status
!git commit -m "initialize dvc"

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

[main f37c756] initialize dvc
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


DVC permet de stocker les données localement ou sur cloud. Il est compatible avec plusieurs services cloud en particulier, avec tous les services de stockage basés sur s3. Dans ce tutoriel, on utilise un bucket s3 hébergé sur minIO. On commence avec un bucket vide.

In [6]:
!dvc remote add --default dvc-remote s3://mbenxsalha/diffusion/dvc
!dvc remote modify dvc-remote endpointurl https://minio.lab.sspcloud.fr
!dvc remote list

Setting 'dvc-remote' as a default remote.
dvc-remote	s3://mbenxsalha/diffusion/dvc


On importe les données et le code sur la mémoire locale.

In [7]:
!mc cp s3/mbenxsalha/diffusion/DiamondsPrices.csv .
!mc cp s3/mbenxsalha/diffusion/train.py .
!ls

...n/train.py: 1.26 KiB / 1.26 KiB ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.14 KiB/s 0s]11;?\DiamondsPrices.csv  README.md  train.py


La commande ```dvc add``` (équivalente à git add) indique à dvc le fichier ou le dossier à suivre en enregistrant ses versions. Pour ce faire, dvc crée un fichier .dvc (DiamondsPrices.csv.dvc dans notre cas) pour suivre les versions. 

In [8]:
!dvc add DiamondsPrices.csv

⠋ Checking graph                                                   ⠋ Checking graph
Adding...                                                                       
!
  0% Checking cache in '/home/onyxia/work/data-version-control/.dvc/cache'| |0/?
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]

!

  0%|          |DiamondsPrices.csv                 0.00/? [00:00<?,        ?B/s]

                                                                                
100% Adding...|████████████████████████████████████████|1/1 [00:00, 11.35file/s]

To track the changes with git, run:

	git add .gitignore DiamondsPrices.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


La commande ```dvc add``` ajoute le fichier à ```.gitignore``` implicitement.

In [9]:
!cat .gitignore

/DiamondsPrices.csv


Le code d'entrainement génère deux modèles (LM_model.pkl et RF_model.pkl) et un fichier ```metrics.csv``` contenant les résultats.

In [10]:
!python train.py
!ls

DiamondsPrices.csv	LM_model.pkl  README.md     train.py
DiamondsPrices.csv.dvc	metrics.csv   RF_model.pkl


On commande dvc de suivre les fichiers nouvellement créés. On note que la taille des fichiers .dvc est beaucoup plus petite que celle des fichiers originaux. Cela permet de déposer des fichiers de très grande taille sur github.

In [11]:
!dvc add metrics.csv LM_model.pkl RF_model.pkl
!ls -lh

⠋ Checking graph                                                   ⠋ Checking graph
  0% Adding...|                                      |0/3 [00:00<?,     ?file/s]
!
  0% Checking cache in '/home/onyxia/work/data-version-control/.dvc/cache'| |0/?
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]

!

  0%|          |metrics.csv                        0.00/? [00:00<?,        ?B/s]

                                                                                
                                                                                
!
  0% Checking cache in '/home/onyxia/work/data-version-control/.dvc/cache'| |0/?
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]

!

  0%|          |LM_model.pkl                       0.00/? [00:00<?,        ?B/s]

       

On sauvegarde notre travail sur github et on tag cette première version pour pouvoir la réviser facilement après.

In [12]:
!git add .
!git commit -m "first version"
!git tag -a "v1.0" -m "first version"
!git push

[main 13a0c5b] first version
 7 files changed, 69 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 DiamondsPrices.csv.dvc
 create mode 100644 LM_model.pkl.dvc
 create mode 100644 RF_model.pkl.dvc
 create mode 100644 metrics.csv.dvc
 create mode 100644 train.py


Dans le bucket, 4 dossiers sont créés, un pour chaque fichier suivi. Chaque dossier contient toutes les versions du fichier.

In [16]:
!mc ls s3/mbenxsalha/diffusion/dvc

]11;?\[2022-11-25 14:42:22 UTC]    39B STANDARD .keep
[2022-11-25 16:05:00 UTC]     0B 43/
[2022-11-25 16:05:00 UTC]     0B 51/
[2022-11-25 16:05:00 UTC]     0B c5/
[2022-11-25 16:05:00 UTC]     0B eb/


In [17]:
import pandas as pd
df = pd.read_csv("DiamondsPrices.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


Ensuite, on modifie le code d'entrainement anisi que les features utilisés.

In [24]:
!python train.py 

In [25]:
import pandas as pd
df = pd.read_csv("DiamondsPrices.csv")
df.head(2)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price
0,0,0.23,Ideal,E,SI2,61.5,55.0,326
1,1,0.21,Premium,E,SI1,59.8,61.0,326


On exécute la même séquence de commandes : dvc add, git add, git commit, git push

In [26]:
!dvc add DiamondsPrices.csv LM_model.pkl RF_model.pkl metrics.csv

⠋ Checking graph                                                   ⠋ Checking graph
  0% Adding...|                                      |0/4 [00:00<?,     ?file/s]
!
  0% Checking cache in '/home/onyxia/work/data-version-control/.dvc/cache'| |0/?
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]

!

  0%|          |DiamondsPrices.csv                 0.00/? [00:00<?,        ?B/s]

                                                                                
                                                                                
!
  0% Checking cache in '/home/onyxia/work/data-version-control/.dvc/cache'| |0/?
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]

!

  0%|          |LM_model.pkl                       0.00/? [00:00<?,        ?B/s]

       

In [27]:
!git add .
!git commit -m "second version"
!git tag -a "v2.0" -m "second version"
!git push

[main 2a2b5b4] second version
 6 files changed, 58 insertions(+), 8 deletions(-)
 create mode 100644 .ipynb_checkpoints/train-checkpoint.py


Maintenant, on peut examiner toutes les versions facilement avec git checkout et dvc checkout. En effet, dvc va importer la version souhaitée des données à partir du stockage externe et recouvrir (overwrite) les fichiers locaux. 

In [31]:
!git checkout v1.0
!dvc checkout
metrics_v1 = pd.read_csv("metrics.csv")
metrics_v1.head()

Previous HEAD position was 2a2b5b4 second version
HEAD is now at 13a0c5b first version
  0% Checkout|                                       |0/4 [00:00<?,     ?file/s]
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-c0.00/137M [00:00<?,        ?B/s]
 86%|████████▌ |/home/onyxia/work/data-versi117M/137M [00:00<00:00,    1.23GB/s]
 25% Checkout|████████▊                          |1/4 [00:00<00:00,  7.12file/s]
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-co0.00/844 [00:00<?,        ?B/s]
                                                                                
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-0.00/2.34M [00:00<?,        ?B/s]
 75% Checkout|██████████████████████████▎        |3/4 [00:00<00:00, 12.78file/s]
!
  0%|         

,Unnamed: 0,RMSE
0,LM,1104.462959
1,RF,559.337674


In [32]:
!git checkout v2.0
!dvc checkout
metrics_v2 = pd.read_csv("metrics.csv")
metrics_v2.head()

Previous HEAD position was 13a0c5b first version
HEAD is now at 2a2b5b4 second version
  0% Checkout|                                       |0/4 [00:00<?,     ?file/s]
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-0.00/1.99M [00:00<?,        ?B/s]
                                                                                
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-c0.00/49.0 [00:00<?,        ?B/s]
                                                                                
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|          |/home/onyxia/work/data-version-co0.00/796 [00:00<?,        ?B/s]
                                                                                
!
  0%|          |/home/onyxia/work/data-version-cont0.00/? [00:00<?,        ?B/s]
  0%|         

,Unnamed: 0,RMSE
0,LM,1160.115737
1,RF,544.569143


# Conclusion
- DVC implémente l'approche ***data as code*** en se basant sur des commandes équivalentes à celles de Git. Il est complémentaire à Git.
- Il permet d'enregistrer les versions de données à suivre dans un stockage local ou externe.
- Il n'offre pas une interface graphique pour explorer les fichiers.
- Son API permet de lire des données à partir du stockage et les mettre dans la mémoire vive directement. Mais elle ne permet pas d'écrire ou de suivre des objets situées dans la mémoire vive sur le remote storage (cela est possible sur lakefs).
- verdict (personnel) : DVC for the win!